In [3]:
import nltk
nltk.download("punkt")

import evaluate
import json
import pandas as pd
import torch as tt

from ast import literal_eval
from datasets import load_dataset, Dataset
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import PreTrainedModel, PreTrainedTokenizer
from typing import Any, Dict
from tqdm import tqdm_notebook

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# models:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("ai-forever/ruT5-base")
model = model.to(tt.device("cuda:0"))

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[nltk_data] Downloading package wordn

In [5]:
df_ege = pd.read_excel("../race-ru-tf/EgeEvalDataset.xlsx", index_col="index")

In [6]:
df_ege.head()

,reading_text,question,right_answer,distractors
index,,,,
0,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Полонез вызвал у автора желание заплакать и с...,"[' Рассказчик был сиротой.', ' В детстве эта м..."
1,"Мы ехали берегом Лены на юг, а зима догоняла ...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,"Подобравшись ближе к берегу, козы бросились к...","[' Собеседник рассказчика, Сокольский, сомнева..."
2,"Воспалённое состояние Поли, а главное, её сби...",Какое высказывание СООТВЕТСТВУЕТ тексту?,Автор письма хранит подарок девочки.,[' Родион встретил девочку перед наступлением....
3,"Все мы любили «классного», хотя нельзя сказат...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Ребята сразу невзлюбили своего классного руко...,[' Белый билет не давал учителю возможности уй...
4,В суровые военные годы во время бомбёжки моя ...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Бабушка рассказчицы долгое время работала в т...,"[' Убежище, в котором укрывалась рассказчица, ..."


In [7]:
df_ege["distractors"] = df_ege["distractors"].apply(literal_eval)

In [8]:
MAX_LEN = 69

In [9]:
ege_dataset = Dataset.from_pandas(df_ege)

In [10]:
def to_dg_format(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    dataset_processed = []
    item_id = 0

    for item in dataset:
        new_item = {
            "item_id": item["index"],
            "passage": item["reading_text"],
            "question": item["question"],
            "distractors": ';'.join(item["distractors"]),
            "right_answer": item["right_answer"]
        }
        dataset_processed.append(new_item)

    return dataset_processed


def to_dg_format_final(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    new_dataset = []

    for item in dataset:
        new_item = {
            "item_id": item["item_id"],
            "inp": f'{item["passage"]} ВОПРОС: {item["question"]} ПРАВИЛЬНЫЙ ОТВЕТ: {item["right_answer"]} НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА: ',
            "outp": item["distractors"],
            "outp_len": len(tokenizer(item["distractors"])["input_ids"])
        }
        new_dataset.append(new_item)

    return new_dataset

In [11]:
ege_dataset = Dataset.from_list(to_dg_format_final(to_dg_format(ege_dataset)))

In [12]:
MAX_LEN = 69 # 0.99 quantile from MuSeRC train dataset

In [13]:
def get_metric_inputs_seq2seq(
    input_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:
    input_batch_ = tokenizer(
        input_batch,
        return_tensors="pt",
        padding=True
    )["input_ids"].to(tt.device("cuda:0"))

    with tt.no_grad():
        output_batch = model.generate(input_batch_, max_length=MAX_LEN)

    output = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in tokenizer.batch_decode(
            output_batch)
    ]
    
    del input_batch_
    del output_batch
    tt.cuda.empty_cache()

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

def compute_metrics_on_dataset_seq2seq(
    dataset: Dataset, model: PreTrainedModel=model,
    tokenizer: PreTrainedTokenizer=tokenizer
) -> pd.DataFrame:
    batch_size = 1

    n_steps = (len(dataset) // batch_size) + 1
    metrics = []

    for i in tqdm_notebook(range(n_steps), total=n_steps):
        slice = dataset[i*batch_size:(i+1)*batch_size]
        if slice["inp"]:
            output = get_metric_inputs_seq2seq(slice["inp"], model, tokenizer)
            distractors = [
                item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip()
                for item in slice["outp"]
            ]
            if len(distractors[0]) > 0:
                metric = compute_metrics(output, distractors)
                metrics.append({
                    "item_id": slice["item_id"][0],
                    "inp": slice["inp"][0],
                    "distractors": distractors[0],
                    "output": output[0],
        
                    "bleu": metric["bleu"]["bleu"],
                    "sbleu": metric["sbleu"]["score"],
                    "rouge1": metric["rouge"]["rouge1"],
                    "rouge2": metric["rouge"]["rouge2"],
                    "rougeL": metric["rouge"]["rougeL"],
                    "rougeLsum": metric["rouge"]["rougeLsum"],
                    "meteor": metric["meteor"]["meteor"],
                })

    return pd.DataFrame(metrics)

METRIC_COLS = [
    "bleu", "sbleu", "rouge1", "rouge2",
    "rougeL", "rougeLsum", "meteor"
]

In [14]:
metrics_ege = compute_metrics_on_dataset_seq2seq(ege_dataset)

/tmp/ipykernel_4177/3452039049.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n_steps), total=n_steps):


  0%|          | 0/56 [00:00<?, ?it/s]

In [15]:
metrics_ege.to_csv("metrics_baseline_ege_t5.csv", sep=';')

In [16]:
metrics_ege[METRIC_COLS].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,55.0,55.000000,55.0,55.0,55.0,55.0,55.000000
mean,0.0,0.543768,0.0,0.0,0.0,0.0,0.028705
std,0.0,0.294495,0.0,0.0,0.0,0.0,0.017407
min,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.420501,0.0,0.0,0.0,0.0,0.015038
50%,0.0,0.579998,0.0,0.0,0.0,0.0,0.023364
75%,0.0,0.720242,0.0,0.0,0.0,0.0,0.040254
max,0.0,1.509525,0.0,0.0,0.0,0.0,0.069832
